# Making an LSTM with Pytorch
### By: Lela Bones and Adam Jump

## Resources
   - [Pytorch Documentation](https://pytorch.org/docs/stable/index.html)
   - [Sine LSTM Example](https://github.com/pytorch/examples/tree/master/time_sequence_prediction)
   - [Noisy Sine LSTM Example](https://gist.github.com/spro/ef26915065225df65c1187562eca7ec4)
   - [LSTM tutorial](https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/recurrent_neural_network/main.py#L38-L56)

### Goal
------------------------
In this notebook we aim to dissect the Pytorch library, build a LSTM model, and test in on sample data.
Then we plan to train it on our brainwave data and run a thorough parameter test to determine what the best parameters are for brainwave data.

### Step One: Generating toy data


In [10]:
#Bringing in libraries we plan on using
import numpy as np
import torch

np.random.seed(2)

T = 20 #number we stretch/shrink our cos wave by
L = 1000 #number of inputs
N = 100 #shape of input

#Creating an empty array that is N X L
x = np.empty((N, L), 'float64')
#APopulates array with X variables
x[:] = np.array(range(L)) + np.random.randint(-4 * T, 4 * T, N).reshape(N, 1)
#Performing the cos function on the data and normalizing it
data = np.cos(x/1/T).astype('float64')
#saving the data in a pickle file
torch.save(data, open('traindata.pt', 'wb'))

### Step Two: Make an LSTM

This is a very basic implemenation of an LSTM, we plan on making this more robust in the future.

In [ ]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.01

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out



Epoch [1/2], Step [100/600], Loss: 0.8510
Epoch [1/2], Step [200/600], Loss: 0.2789
Epoch [1/2], Step [300/600], Loss: 0.2597
Epoch [1/2], Step [400/600], Loss: 0.0814


### Step Three: Train LSTM

In [ ]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

### Step Four: Predict using trained LSTM

### Step Five: Visualizing LSTM

In [14]:
input = torch.from_numpy(data[3:, :-1])
target = torch.from_numpy(data[3:, 1:])
input.shape
target.shape

torch.Size([12321, 8])